In [13]:
######################################## IMPORTS #####################################

import pygame
import numpy as np
from random import randrange
import random
import os
import json

#####################################################################################

############################### CONSTANTS ##################################
# Initialisation de pygame et de la fenêtre
pygame.init()
pygame.font.init()

# Couleurs
sombre = True

def mode_couleur():
    if sombre:
        return {
            'FOND': (51, 51, 51),  # Charbon profond et sombre
            'VIVANTE': (88, 129, 87),  # Sarcelle douce
            'MORTE': (22, 24, 28),  # Presque noir avec une touche de profondeur
            'FOND_PANEL': (24, 26, 32),  # Gris ardoise foncé

            'BOUTON': (76, 112, 75),  # Bleu-gris foncé atténué
            'BOUTON_SURVOL': (108, 141, 107),  # Variante légèrement plus claire
            'TEXTE': (210, 220, 230) # Doux blanc cassé
          }
    else:
        return {
        'FOND': (248, 235, 220),  # Beige chaud comme base
        'VIVANTE': (106, 153, 78),  # Vert sage adouci
        'MORTE': (248, 235, 220),  # Beige légèrement plus foncé
        'FOND_PANEL': (232, 220, 202),  # Beige très clair

        'BOUTON': (143, 176, 125),  # Vert sage clair
        'BOUTON_SURVOL': (162, 189, 147),  # Vert sage plus clair
        'TEXTE': (67, 90, 49)  # Vert foncé pour le texte
        }

def toggle_mode():
    global  sombre
    sombre = not sombre

POLICE_BOUTON = pygame.font.SysFont('montserrat', 32)
POLICE_TOGGLE = pygame.font.SysFont('montserrat', 14)

# Initialisation du jeu
TAILLE_CELLULE = 7
DIMENSION = 100

############################### FONCTIONS DE BASES ##################################

def grille(hauteur, largeur, etat, etat_futur, cellule, fenetre):
    """ Calcule et dessine la nouvelle grille """
    protocole(hauteur, largeur, etat, etat_futur),draw(hauteur, largeur, cellule, etat, fenetre)
    pygame.display.flip()  # Rafraîchit l'affichage


def initialisation(dimension=DIMENSION):
    """ Initialisation | Renvoie la hauteur, la largeur et les matrices des etats """
    hauteur, largeur = dimension,dimension
    
    # Matrice qui gèrera l'aspect graphique des cellules (rectangles)
    cellule = [[None for _ in range(hauteur)] for _ in range(largeur)]
    # Matrice qui contient l'état actuel de chaque cellules [ 0=morte / 1=vivante], elles sont toutes mortes au début
    etat = [[0 for _ in range(hauteur)] for _ in range(largeur)]
    # Matrice qui contient le prochain état de chaque cellule (la matrice précedente prendra sa valeur)
    etat_futur = [[0 for _ in range(hauteur)] for _ in range(largeur)]
    
    for y in range(hauteur):
        for x in range(largeur):
            etat[x][y] = 0
            etat_futur[x][y] = 0
            # Création des rectangles
            cellule[x][y] = pygame.Rect(x*TAILLE_CELLULE, y*TAILLE_CELLULE, TAILLE_CELLULE, TAILLE_CELLULE) 
            
    # Place environ 25% de cellules vivantes
    for i in range(largeur * hauteur // 4):
        etat[randrange(largeur)][randrange(hauteur)] = 1
        
    return hauteur, largeur, cellule, etat, etat_futur

def initialisation_aleatoire():
    """ Initialisation aléatoire de la grille : intervalle régulable """
    dimension = random.randint(50, 300)
    hauteur, largeur, cellule, etat, etat_futur = initialisation(dimension)
    return dimension, hauteur, largeur, cellule, etat, etat_futur

def protocole(hauteur, largeur, etat, etat_futur):
    """ Calcule et applique les règles """
    for y in range(hauteur):
        for x in range(largeur):
            nombre_voisins = compte_voisins_vivant(x, y, hauteur, largeur, etat)
            # Applique les règles de survie et de naissance des cellules
            if etat[x][y] == 1 and nombre_voisins < 2:
                etat_futur[x][y] = 0
            elif etat[x][y] == 1 and nombre_voisins > 3:
                etat_futur[x][y] = 0
            elif etat[x][y] == 1 and (nombre_voisins == 2 or nombre_voisins == 3):
                etat_futur[x][y] = 1
            elif etat[x][y] == 0 and nombre_voisins == 3:
                etat_futur[x][y] = 1
        
    for y in range(hauteur):
        for x in range(largeur):
            etat[x][y] = etat_futur[x][y]

def compte_voisins_vivant(x, y, hauteur, largeur, etat):
    """ Calcul et renvoie le nombre de cellules voisines en vie (tableau torique) """
    nb_voisins = 0
    # On definit une matrice contenant les coordonnées de tous les voisins d'une cellule x=0 et y=0
    directions = [(-1, 1), (0, 1), (1, 1), (-1, 0), (1, 0), (-1, -1), (0, -1), (1, -1)]
    
    for dx, dy in directions:
        # on utilise le % pour que le bord gauche soit connecté au bord droit, 
        # et le bord supérieur soit connecté au bord inférieur
        if etat[(x + dx) % largeur][(y + dy) % hauteur] == 1:
            nb_voisins += 1
    return nb_voisins

def draw(hauteur, largeur, cellule, etat, fenetre):
    """ Dessine / Redessine toutes les cellules """
    couleurs = mode_couleur()
    fenetre.fill(couleurs['FOND'])  # Fond
    
    for y in range(hauteur):
        for x in range(largeur):
            couleur = couleurs['VIVANTE']  if etat[x][y] == 1 else couleurs['MORTE']
            pygame.draw.rect(fenetre, couleur, cellule[x][y])
    
    # Définir le panel à droite
    panel_rect = pygame.Rect(TAILLE_CELLULE * largeur, 0, LARGEUR_PANEL_DROIT, TAILLE_CELLULE * hauteur)
    panel_surface = pygame.Surface(panel_rect.size, pygame.SRCALPHA)
    # Dessiner le panel
    panel_surface.fill(couleurs['FOND_PANEL'])
    fenetre.blit(panel_surface, panel_rect) # Copie le rect et la surface sur l'écran.

    # Boutons
    draw_button(fenetre, "SAVE", (TAILLE_CELLULE * largeur + 50, 50, 300, 50))
    draw_button(fenetre, "LOAD", (TAILLE_CELLULE * largeur + 50, 150, 300, 50))
    draw_button(fenetre, "RESET", (TAILLE_CELLULE * largeur + 50, 250, 300, 50))
    
    mode_texte = "Mode Clair" if sombre else "Mode Sombre"
    draw_button(fenetre, mode_texte,
                (TAILLE_CELLULE * largeur + LARGEUR_PANEL_DROIT - 90, 10, 70, 25),
                is_toggle=True)

#####################################################################################

############################### FONCTIONS SUPLEMENTAIRES ############################

def modifie_etat_cellule(x, y, hauteur, largeur, etat, taille_cellule=TAILLE_CELLULE):
    """ Modifie l'état de la cellule aux coordonnées x,y (largeur, hauteur) à chaque clic de souris """
    if x <= largeur * TAILLE_CELLULE and y <= hauteur * taille_cellule:
        # Calcule les indices de la cellule cliquée
        colonne = x // taille_cellule
        ligne = y // taille_cellule
        
        # Inverse l'état de la cellule
        etat[colonne][ligne] = 1 if etat[colonne][ligne] == 0 else 0
        
        print(f"Cellule à ({colonne}, {ligne}) maintenant {'vivante' if etat[colonne][ligne] == 1 else 'morte'}")
    

##### BOUTONS

def draw_button(surface, text, rect, font=POLICE_BOUTON, is_toggle=False):
    """ Bouton avec survol """
    couleurs = mode_couleur()
    mouse_pos = pygame.mouse.get_pos()
    button_rect = pygame.Rect(rect)
    
    if not is_toggle:
        couleur_curr = couleurs['BOUTON_SURVOL'] if button_rect.collidepoint(mouse_pos) else couleurs['BOUTON']
        pygame.draw.rect(surface, couleur_curr, button_rect, border_radius=8)
    
    police_curr = POLICE_TOGGLE if is_toggle else font
    text_surface = police_curr.render(text, True, couleurs['TEXTE'])
    text_rect = text_surface.get_rect(center=button_rect.center)
    surface.blit(text_surface, text_rect)
    
    return button_rect

##### SAUVEGARDE | CHARGEMENT

def sauvegarde(hauteur, largeur, etat):
    """ Sauvergarder l'état de la grille """
    try:
        os.makedirs('saves', exist_ok=True)
        filename = f'saves/game_of_life_save_{pygame.time.get_ticks()}.json'
        
        save_data = {
            'hauteur': hauteur,
            'largeur': largeur,
            'etat': etat
        }
        
        with open(filename, 'w') as f:
            json.dump(save_data, f)
        return filename
    
    except Exception as e:
        print(f"Enregistrement échoué: {e}")
        return None

def charger():
    """ Charger la sauvegarde de jeu la plus récente """
    try:
        saves = []
        for fich in os.listdir('saves'):
            if fich.startswith("game_of_life_save_") and fich.endswith(".json"):
                saves.append(fich)
        print("saved files: ", saves)
        if not saves:
            return None
        
        dernier_save = max(saves, key=lambda x: int(x.split('_')[-1].split('.')[0]))
        print(f'dernier_save: {dernier_save}')
        filepath = os.path.join('saves', dernier_save)
        
        with open(filepath, 'r') as fich:
            save_data = json.load(fich)
        
        return save_data['hauteur'], save_data['largeur'], save_data['etat']
    
    except Exception as e:
        print(f"Le chargement a échoué: {e}")
        return None

#####################################################################################

##################################### LANCEMENT #####################################

hauteur, largeur, cellule, etat, etat_futur = initialisation()
#dimension, hauteur, largeur, cellule, etat, etat_futur = initialisation_aleatoire() # pour initialiser aléatoirement la grille

# Creation fenetre : elle sera de largeur : taille_cellule*largeur et hauteur : taille_cellule*hauteur  
LARGEUR_PANEL_DROIT = 400
fenetre = pygame.display.set_mode((TAILLE_CELLULE*largeur + LARGEUR_PANEL_DROIT, TAILLE_CELLULE*hauteur))
pygame.display.set_caption("Projet : Jeu de la vie")  
    
# Boucle de jeu
running = True
clock = pygame.time.Clock() # ???

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:  # Vérifie si un clic de souris a eu lieu
            pos_x, pos_y = pygame.mouse.get_pos()  # Récupère les coordonnées de la souris
            
            save_button = pygame.Rect(TAILLE_CELLULE*largeur + 50, 50, 300, 50)
            load_button = pygame.Rect(TAILLE_CELLULE*largeur + 50, 150, 300, 50)
            reset_button = pygame.Rect(TAILLE_CELLULE*largeur + 50, 250, 300, 50)
            toggle_button = pygame.Rect(TAILLE_CELLULE*largeur + LARGEUR_PANEL_DROIT - 60, 10, 50, 30)
            
            if save_button.collidepoint(pos_x, pos_y):
                sauvegarde(hauteur, largeur, etat)

            elif load_button.collidepoint(pos_x, pos_y):
                jeu_charge = charger()
                print(jeu_charge)
                if jeu_charge:
                    hauteur, largeur, etat = jeu_charge
                    etat_futur = [[0 for _ in range(hauteur)] for _ in range(largeur)]
                    cellule = [[pygame.Rect(x*TAILLE_CELLULE, y*TAILLE_CELLULE, TAILLE_CELLULE-1, TAILLE_CELLULE-1) 
                                for y in range(hauteur)] for x in range(largeur)]
            
            elif reset_button.collidepoint(pos_x, pos_y):
                hauteur, largeur, cellule, etat, etat_futur = initialisation()
            
            elif toggle_button.collidepoint(pos_x, pos_y):
                toggle_mode()
            else:
                modifie_etat_cellule(pos_x, pos_y, hauteur, largeur, etat)
            

    # Mise à jour et affichage de la grille
    grille(hauteur, largeur, etat, etat_futur, cellule, fenetre)
    #pygame.time.delay(20)  # Vitesse de l'animation (combien de miliseconde entre chaque changement d'état)
    clock.tick(10)
# Fermeture de pygame
pygame.quit()

#####################################################################################